<a href="https://colab.research.google.com/github/cbonnin88/E-commerce-Retention-Engine/blob/main/theLook_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import auth
from google.cloud import bigquery
import polars as pl
import plotly.express as px
import plotly.graph_objects as go

In [2]:
auth.authenticate_user()
project_id = 'poetic-maxim-486609-e5'
client = bigquery.Client(project=project_id)

In [3]:
master_query = """
SELECT
  user_id,
  event_type,
  session_id,
  created_at
FROM `bigquery-public-data.thelook_ecommerce.events`
"""

In [4]:
df_thelook = pl.from_pandas(client.query(master_query).to_dataframe())

In [5]:
df_thelook.head()

user_id,event_type,session_id,created_at
i64,str,str,"datetime[μs, UTC]"
null,"""cancel""","""5807e795-c47b-45cf-bede-6517a4…",2023-10-04 11:14:00 UTC
null,"""cancel""","""db6251b9-5004-4e35-bee8-d2cbd3…",2021-05-06 17:33:00 UTC
null,"""cancel""","""86a1ef31-bf8e-4a63-b1a5-c711c1…",2020-03-25 08:20:00 UTC
null,"""cancel""","""38a88044-a9d7-4d17-9344-8587bd…",2023-10-02 13:01:00 UTC
null,"""cancel""","""b034c9ae-e20a-4d3c-b417-01d245…",2022-12-29 02:56:00 UTC


# **Funnel Aggregation**

In [6]:
funnel = df_thelook.group_by('event_type').agg(
    pl.col('user_id').n_unique().alias('unique_users')
).sort('unique_users', descending=True)

In [7]:
display(funnel)

event_type,unique_users
str,u32
"""product""",80081
"""department""",80081
"""cart""",80081
"""purchase""",80080
"""home""",63184
"""cancel""",1


In [8]:
fig_funnel = go.Figure(go.Funnel(
    y=funnel['event_type'].to_list(),
    x=funnel['unique_users'].to_list(),
    textinfo='value+percent initial'
))

fig_funnel.update_layout(title_text='TheLook: User Conversion Funnel')
fig_funnel.show()

# **Cohort Analysis**

- This is the **"Holy Grail"** of Product Analytics. We want to see: *Of the users who joined in Month X, what % returned in Month X+1?*

In [9]:
# Creating Cohort logic in Polars
# 1. Get first purchase date per user
df_orders = df_thelook.filter(pl.col('event_type') == 'purchase')

display(df_orders.head())

user_id,event_type,session_id,created_at
i64,str,str,"datetime[μs, UTC]"
59798,"""purchase""","""fb663430-240d-4873-ae91-908c87…",2024-12-21 11:44:55 UTC
84852,"""purchase""","""ffe5843c-76f4-4ef2-8a25-30b7d2…",2025-03-21 16:47:42 UTC
91237,"""purchase""","""f2843b7b-a624-4176-8bfa-795c98…",2026-01-05 07:49:10 UTC
908,"""purchase""","""6b198882-3801-442f-868f-9dde37…",2021-02-15 11:55:43 UTC
96846,"""purchase""","""81e042e8-ebe6-42ef-87d8-9a361e…",2025-08-25 02:18:51 UTC


In [10]:
# 2. Define the Cohort (The month of the user's FIRST purchase)
df_orders = df_orders.with_columns([
    pl.col('created_at').dt.truncate('1mo').alias('order_month')
])

user_cohorts = df_orders.group_by('user_id').agg(
    pl.col('order_month').min().alias('cohort_month')
)

display(user_cohorts.head())

user_id,cohort_month
i64,"datetime[μs, UTC]"
48659,2026-02-01 00:00:00 UTC
62459,2022-12-01 00:00:00 UTC
24620,2025-09-01 00:00:00 UTC
10416,2023-03-01 00:00:00 UTC
30705,2021-02-01 00:00:00 UTC


In [11]:
# 3. Join the 'Birth Month' back to all orders
df_cohort = df_orders.join(user_cohorts, on='user_id')

display(df_cohort.head())

user_id,event_type,session_id,created_at,order_month,cohort_month
i64,str,str,"datetime[μs, UTC]","datetime[μs, UTC]","datetime[μs, UTC]"
59798,"""purchase""","""fb663430-240d-4873-ae91-908c87…",2024-12-21 11:44:55 UTC,2024-12-01 00:00:00 UTC,2024-12-01 00:00:00 UTC
84852,"""purchase""","""ffe5843c-76f4-4ef2-8a25-30b7d2…",2025-03-21 16:47:42 UTC,2025-03-01 00:00:00 UTC,2022-08-01 00:00:00 UTC
91237,"""purchase""","""f2843b7b-a624-4176-8bfa-795c98…",2026-01-05 07:49:10 UTC,2026-01-01 00:00:00 UTC,2026-01-01 00:00:00 UTC
908,"""purchase""","""6b198882-3801-442f-868f-9dde37…",2021-02-15 11:55:43 UTC,2021-02-01 00:00:00 UTC,2021-02-01 00:00:00 UTC
96846,"""purchase""","""81e042e8-ebe6-42ef-87d8-9a361e…",2025-08-25 02:18:51 UTC,2025-08-01 00:00:00 UTC,2025-08-01 00:00:00 UTC


In [12]:
# 4. Calculate the 'Cohort Index' (Months since first purchase)
# Logic: (Year Diff * 12) + Month Diff
df_cohort = df_cohort.with_columns([
    ((pl.col('order_month').dt.year() - pl.col('cohort_month').dt.year()) * 12 +
     (pl.col('order_month').dt.month() - pl.col('cohort_month').dt.month())).alias('cohort_index')
])

display(df_cohort.head())

user_id,event_type,session_id,created_at,order_month,cohort_month,cohort_index
i64,str,str,"datetime[μs, UTC]","datetime[μs, UTC]","datetime[μs, UTC]",i32
59798,"""purchase""","""fb663430-240d-4873-ae91-908c87…",2024-12-21 11:44:55 UTC,2024-12-01 00:00:00 UTC,2024-12-01 00:00:00 UTC,0
84852,"""purchase""","""ffe5843c-76f4-4ef2-8a25-30b7d2…",2025-03-21 16:47:42 UTC,2025-03-01 00:00:00 UTC,2022-08-01 00:00:00 UTC,31
91237,"""purchase""","""f2843b7b-a624-4176-8bfa-795c98…",2026-01-05 07:49:10 UTC,2026-01-01 00:00:00 UTC,2026-01-01 00:00:00 UTC,0
908,"""purchase""","""6b198882-3801-442f-868f-9dde37…",2021-02-15 11:55:43 UTC,2021-02-01 00:00:00 UTC,2021-02-01 00:00:00 UTC,0
96846,"""purchase""","""81e042e8-ebe6-42ef-87d8-9a361e…",2025-08-25 02:18:51 UTC,2025-08-01 00:00:00 UTC,2025-08-01 00:00:00 UTC,0


In [13]:
# 5. Aggregate: Count unique users per Cohort and Index
cohort_counts = df_cohort.group_by(['cohort_month','cohort_index']).agg(
    pl.col('user_id').n_unique().alias('user_count')
).sort(['cohort_month','cohort_index'])

display(cohort_counts.head())

cohort_month,cohort_index,user_count
"datetime[μs, UTC]",i32,u32
2019-01-01 00:00:00 UTC,0,11
2019-01-01 00:00:00 UTC,16,1
2019-01-01 00:00:00 UTC,18,1
2019-01-01 00:00:00 UTC,19,1
2019-01-01 00:00:00 UTC,21,1


In [14]:
# 6. Transform to Retention Percentage
# Getting the size of the initial cohort (index 0)
cohort_sizes = cohort_counts.filter(pl.col('cohort_index') == 0).select([
    pl.col('cohort_month'),
    pl.col('user_count').alias('initial_size')
])

retention_df = cohort_counts.join(cohort_sizes, on='cohort_month')
retention_df = retention_df.with_columns(
    (pl.col('user_count') / pl.col('initial_size')).alias('retention_rate')
)

display(retention_df.head())

cohort_month,cohort_index,user_count,initial_size,retention_rate
"datetime[μs, UTC]",i32,u32,u32,f64
2019-01-01 00:00:00 UTC,0,11,11,1.0
2019-01-01 00:00:00 UTC,16,1,11,0.090909
2019-01-01 00:00:00 UTC,18,1,11,0.090909
2019-01-01 00:00:00 UTC,19,1,11,0.090909
2019-01-01 00:00:00 UTC,21,1,11,0.090909


In [15]:
# 7. Pivot for the heatmap
# I limited to the last 12 months for readability

final_pivot = retention_df.to_pandas().pivot(
    index='cohort_month',
    columns='cohort_index',
    values='retention_rate'
).iloc[-12:,:12] # Last 12 cohorts, first 12 months

display(final_pivot.head())

cohort_index,0,1,2,3,4,5,6,7,8,9,10,11
cohort_month,,,,,,,,,,,,
2025-03-01 00:00:00+00:00,1.0,0.053961,0.037887,0.042480,0.038462,0.044202,0.044776,0.048794,0.031573,0.053961,0.044776,0.025832
2025-04-01 00:00:00+00:00,1.0,0.072818,0.040022,0.045581,0.046693,0.045025,0.050584,0.050028,0.045581,0.043357,0.022235,NaN
2025-05-01 00:00:00+00:00,1.0,0.059349,0.046744,0.050420,0.048845,0.043067,0.035189,0.050945,0.048319,0.025735,NaN,NaN
2025-06-01 00:00:00+00:00,1.0,0.066482,0.052632,0.062050,0.060942,0.060388,0.054294,0.054848,0.027701,NaN,NaN,NaN
2025-07-01 00:00:00+00:00,1.0,0.080127,0.062203,0.060622,0.050606,0.055351,0.053242,0.031629,NaN,NaN,NaN,NaN


In [16]:
fig_cohort = px.imshow(
    final_pivot,
    labels=dict(x='Months Since First Purchase',y='Cohort Month',color='Retention Rate'),
    x=final_pivot.columns,
    y=final_pivot.index.astype(str),
    color_continuous_scale='Viridis',
    text_auto='.1%'
)

fig_cohort.update_layout(
    title='User Retention Cohorts: % of Users Returning MoM',
    xaxis_title='Month Index (0= First Purchase)',
    yaxis_title= 'User Start Month'
)

fig_cohort.show()

# **ARPPU (Average Revenue Per Paying User)**

- This tracks the *"depth"* of your monetization. Unlike AOV (which is per order), ARPPU is per **customer.**

In [17]:
# Because 'sale-price' is in the 'order_items' table and not in the events, I used the dbt Mart
query_arppu = """
SELECT
  user_id,
  sale_price,
  DATE_TRUNC(created_at,MONTH) AS month
FROM `bigquery-public-data.thelook_ecommerce.order_items`
WHERE status NOT IN ('Cancelled','Returned')
"""

In [18]:
df_arppu_raw = pl.from_pandas(client.query(query_arppu).to_dataframe())

In [19]:
display(df_arppu_raw.head())

user_id,sale_price,month
i64,f64,"datetime[μs, UTC]"
2981,0.49,2020-10-01 00:00:00 UTC
93841,1.51,2020-07-01 00:00:00 UTC
57184,1.72,2019-02-01 00:00:00 UTC
93701,1.75,2020-12-01 00:00:00 UTC
93954,1.95,2019-06-01 00:00:00 UTC


In [20]:
# Group by Month and User to get 'Total Spend per User'
user_monthly_spend = df_arppu_raw.group_by(['month','user_id']).agg(
    pl.col('sale_price').sum().alias('total_user_spend')
)

display(user_monthly_spend.head())

month,user_id,total_user_spend
"datetime[μs, UTC]",i64,f64
2023-01-01 00:00:00 UTC,86618,88.0
2023-01-01 00:00:00 UTC,5164,49.450001
2024-07-01 00:00:00 UTC,23815,9.97
2023-02-01 00:00:00 UTC,994,21.200001
2023-10-01 00:00:00 UTC,23392,35.0


In [21]:
# Calculate ARPPU: The average of those user totals per months
arppu_final = user_monthly_spend.group_by('month').agg(
    pl.col('total_user_spend').mean().alias('arppu')
).sort('month')

display(arppu_final.head())

month,arppu
"datetime[μs, UTC]",f64
2019-01-01 00:00:00 UTC,36.7825
2019-02-01 00:00:00 UTC,85.546112
2019-03-01 00:00:00 UTC,70.258667
2019-04-01 00:00:00 UTC,95.792392
2019-05-01 00:00:00 UTC,84.578772


In [22]:
fig_arppu = px.line(
    arppu_final.to_pandas(),
    x='month',
    y='arppu',
    title='Average Revenue Per Paying User (ARPPU) - Monthly Trend',
    markers=True,
    line_shape='spline',
    template='plotly_white'
)

# Adding a median line to show the 'Typical' performance
median_arppu = arppu_final['arppu'].median()
fig_arppu.add_hline(
    y=median_arppu,
    line_dash='dot',
    annotation_text=f'Median: €{median_arppu:.2f}',
    line_color='red'
)

fig_arppu.update_layout(yaxis_tickprefix='€',yaxis_title='ARPPU (€)')
fig_arppu.show()

# **Revenue per Event (RPE)**

- This metric tells you the *"value"* of a single click. It helps Product Managers understand which events (like "Product View" vs. "Add to Cart") are most correlated with high spend.

In [23]:
# 1. Load the two tables from BigQuery

query_events = """
SELECT
  user_id,
  event_type,
  created_at
FROM `bigquery-public-data.thelook_ecommerce.events`
"""

In [24]:
query_revenue ="""
SELECT
  user_id,
  sale_price,
  created_at AS order_at
FROM `bigquery-public-data.thelook_ecommerce.order_items`
WHERE status != 'Cancelled'
"""

In [25]:
df_events = pl.from_pandas(client.query(query_events).to_dataframe())
df_revenue = pl.from_pandas(client.query(query_revenue).to_dataframe())

In [26]:
# 2. Join the data (LEFT JOIN ensures that I keep all the event data)
# Joining on the user_id to see the revenue generated by browinsg users

df_combined = df_events.join(df_revenue, on='user_id',how='left')

In [27]:
# Calculate the average sale price generated per event type
rpe_df = df_combined.group_by('event_type').agg(
    pl.col('sale_price').mean().alias('avg_revenue_impact')
).drop_nulls().sort('avg_revenue_impact',descending=True)

display(rpe_df.head())

event_type,avg_revenue_impact
str,f64
"""home""",59.814145
"""purchase""",59.648368
"""product""",59.590298
"""cart""",59.590298
"""department""",59.590298


In [28]:
fig_rpe = px.bar(
    rpe_df.to_pandas(),
    x='event_type',
    y='avg_revenue_impact',
    title='Financial Impact per Event Type',
    labels = {'avg_revenue_impact':'Avg Sales Price (€)','event_type':'Event'},
    color='avg_revenue_impact',
    text_auto='.2f'

)

fig_rpe.show()